In [2]:
import os
# Change native directory to root
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [4]:
data_test = pd.read_csv('data/extra_test.csv', index_col=0)
TARGET = 'net_target'
y = data_test[TARGET].values

In [9]:
data_test.columns

Index(['Month', 'Hour', 'Day Type', 'cons_target-1', 'Solar Generation [W/kW]',
       'Outdoor Drybulb Temperature [C]', 'Relative Humidity [%]',
       'Diffuse Solar Radiation [W/m2]', 'Direct Solar Radiation [W/m2]',
       'building', 'day_year', 'dayinyear', 'timestamp', 'number',
       'weekday_hour', 'hour_x', 'hour_y', 'month_x', 'month_y', 'weekday_x',
       'weekday_y', 'net', 'net_target', 'net+1', 'cons+1', 'gen+1',
       'net_target-1', 'net_target-1_mean_lag3', 'net_target-1_max_lag3',
       'net_target-1_min_lag3', 'net_target-1_std_lag3', 'cons_target',
       'gen_target', 'cons_target-2', 'gen_target-2',
       'diffuse_solar_radiation+1', 'drybulb_temp+1', 'relative_humidity+1',
       'net_target-23'],
      dtype='object')

In [12]:
data_test.Hour.unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.])

In [7]:
def calc_covariance_matrix(params, data, horizon=24):
    sigma, alpha = params[0], params[1:]
    p = len(alpha)
    T = data.shape[0]
    n_chunks = T // horizon
    cov_matrix = np.zeros((horizon, horizon))
    for chunk in range(n_chunks):
        start = chunk * horizon
        end = start + horizon
        Z = data[start:end, :]
        for i in range(horizon):
            for j in range(horizon):
                cov_matrix[i, j] = sigma**2
                for k in range(1, p+1):
                    if i-k >= 0 and j-k >= 0:
                        cov_matrix[i, j] += alpha[k-1]**2 * cov_matrix[i-k, j-k]
                    if i-k >= 0 and j-k < 0:
                        cov_matrix[i, j] += alpha[k-1]**2 * cov_matrix[i-k, horizon + (j-k)]
                    if i-k < 0 and j-k >= 0:
                        cov_matrix[i, j] += alpha[k-1]**2 * cov_matrix[horizon + (i-k), j-k]
                    for l in range(1, k):
                        if i-l >= 0 and j-k >= 0:
                            cov_matrix[i, j] += 2 * alpha[l-1] * alpha[k-1] * cov_matrix[i-l, j-k]
                        if i-l >= 0 and j-k < 0:
                            cov_matrix[i, j] += 2 * alpha[l-1] * alpha[k-1] * cov_matrix[i-l, horizon + (j-k)]
                        if i-k < 0 and j-l >= 0:
                            cov_matrix[i, j] += 2 * alpha[l-1] * alpha[k-1] * cov_matrix[horizon + (i-k), j-l]
    return cov_matrix

In [42]:
# Define the order of the AR(p) process
p = 2

# Define the initial values for the parameters
initial_params = [1, 0.5, 0.5]

# Define the bounds for the parameters
bounds = [(1e-5, None), (-1, 1), (-1, 1)]

# Perform the optimization
res = minimize(log_likelihood, initial_params, args=(y, p), bounds=bounds, method='L-BFGS-B')

ValueError: shapes (8710,) and (24,24) not aligned: 8710 (dim 0) != 24 (dim 0)

In [33]:
# check if there is nan in y
np.isnan(y).any()

False